In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split

from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
import numpy as np

In [ ]:
holiday_data = pd.read_csv('/content/holidays_events.csv')
oil_data = pd.read_csv('/content/oil.csv')
stores_data = pd.read_csv('/content/stores.csv')
train_data = pd.read_csv('/content/train.csv')
transactions_data = pd.read_csv('/content/transactions.csv')
test_data = pd.read_csv('/content/test.csv')

In [ ]:
holiday_data['date'] = pd.to_datetime(holiday_data['date'])
oil_data['date'] = pd.to_datetime(oil_data['date'])
train_data['date'] = pd.to_datetime(train_data['date'])

Data Preparation

In [ ]:
#Merging train.csv & stores.csv
train_data = pd.merge(train_data,stores_data, on='store_nbr', how='left')

In [ ]:
#Merging train.csv & holiday.csv
train_data = pd.merge(train_data,holiday_data, on='date',how='left')
train_data.rename(columns={'type_y': 'holiday_type', 'type_shop': 'type_shop'}, inplace=True)

In [ ]:
train_data

,id,date,store_nbr,family,sales,onpromotion,city,state,type_x,cluster,holiday_type,locale,locale_name,description,transferred
0,0,2013-01-01,1,AUTOMOTIVE,0.000,0,Quito,Pichincha,D,13,Holiday,National,Ecuador,Primer dia del ano,False
1,1,2013-01-01,1,BABY CARE,0.000,0,Quito,Pichincha,D,13,Holiday,National,Ecuador,Primer dia del ano,False
2,2,2013-01-01,1,BEAUTY,0.000,0,Quito,Pichincha,D,13,Holiday,National,Ecuador,Primer dia del ano,False
3,3,2013-01-01,1,BEVERAGES,0.000,0,Quito,Pichincha,D,13,Holiday,National,Ecuador,Primer dia del ano,False
4,4,2013-01-01,1,BOOKS,0.000,0,Quito,Pichincha,D,13,Holiday,National,Ecuador,Primer dia del ano,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3054343,3000883,2017-08-15,9,POULTRY,438.133,0,Quito,Pichincha,B,6,Holiday,Local,Riobamba,Fundacion de Riobamba,False
3054344,3000884,2017-08-15,9,PREPARED FOODS,154.553,1,Quito,Pichincha,B,6,Holiday,Local,Riobamba,Fundacion de Riobamba,False
3054345,3000885,2017-08-15,9,PRODUCE,2419.729,148,Quito,Pichincha,B,6,Holiday,Local,Riobamba,Fundacion de Riobamba,False
3054346,3000886,2017-08-15,9,SCHOOL AND OFFICE SUPPLIES,121.000,8,Quito,Pichincha,B,6,Holiday,Local,Riobamba,Fundacion de Riobamba,False


In [ ]:
# Add day_of_week to identify weekends
oil_data['day_of_week'] = oil_data['date'].dt.dayofweek  # Monday = 0, Sunday = 6

# Forward fill for weekends (Saturday = 5, Sunday = 6)
mask = (oil_data['day_of_week'] == 5) | (oil_data['day_of_week'] == 6)
oil_data.loc[mask, 'dcoilwtico'] = oil_data['dcoilwtico'].ffill()

# Forward-fill missing oil prices
oil_data['dcoilwtico'] = oil_data['dcoilwtico'].ffill()
oil_data

,date,dcoilwtico,day_of_week
0,2013-01-01,NaN,1
1,2013-01-02,93.14,2
2,2013-01-03,92.97,3
3,2013-01-04,93.12,4
4,2013-01-07,93.20,0
...,...,...,...
1213,2017-08-25,47.65,4
1214,2017-08-28,46.40,0
1215,2017-08-29,46.46,1
1216,2017-08-30,45.96,2


In [ ]:
#Merging train.csv & oil.csv
train_data = pd.merge(train_data, oil_data, on='date',how='left')

In [ ]:
# train_data['dcoilwtico'] = train_data['dcoilwtico'].bfill()
# train_data.rename(columns={'dcoilwtico': 'oil_price'}, inplace=True)
train_data

,id,date,store_nbr,family,sales,onpromotion,city,state,type_x,cluster,...,description,transferred,oil_price,day_of_week,category_type,dayofmonth,dayofweek,quarter,month,year
0,0,2013-01-01,1,AUTOMOTIVE,0.000,0,Quito,Pichincha,D,13,...,Primer dia del ano,False,93.14,1.0,holiday,1,1,1,1,2013
1,1,2013-01-01,1,BABY CARE,0.000,0,Quito,Pichincha,D,13,...,Primer dia del ano,False,93.14,1.0,holiday,1,1,1,1,2013
2,2,2013-01-01,1,BEAUTY,0.000,0,Quito,Pichincha,D,13,...,Primer dia del ano,False,93.14,1.0,holiday,1,1,1,1,2013
3,3,2013-01-01,1,BEVERAGES,0.000,0,Quito,Pichincha,D,13,...,Primer dia del ano,False,93.14,1.0,holiday,1,1,1,1,2013
4,4,2013-01-01,1,BOOKS,0.000,0,Quito,Pichincha,D,13,...,Primer dia del ano,False,93.14,1.0,holiday,1,1,1,1,2013
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3054343,3000883,2017-08-15,9,POULTRY,438.133,0,Quito,Pichincha,B,6,...,Fundacion de Riobamba,False,47.57,1.0,holiday,15,1,3,8,2017
3054344,3000884,2017-08-15,9,PREPARED FOODS,154.553,1,Quito,Pichincha,B,6,...,Fundacion de Riobamba,False,47.57,1.0,holiday,15,1,3,8,2017
3054345,3000885,2017-08-15,9,PRODUCE,2419.729,148,Quito,Pichincha,B,6,...,Fundacion de Riobamba,False,47.57,1.0,holiday,15,1,3,8,2017
3054346,3000886,2017-08-15,9,SCHOOL AND OFFICE SUPPLIES,121.000,8,Quito,Pichincha,B,6,...,Fundacion de Riobamba,False,47.57,1.0,holiday,15,1,3,8,2017


In [ ]:
train_data

,id,date,store_nbr,family,sales,onpromotion,city,state,type_x,cluster,holiday_type,locale,locale_name,description,transferred,oil_price,day_of_week
0,0,2013-01-01,1,AUTOMOTIVE,0.000,0,Quito,Pichincha,D,13,Holiday,National,Ecuador,Primer dia del ano,False,93.14,1.0
1,1,2013-01-01,1,BABY CARE,0.000,0,Quito,Pichincha,D,13,Holiday,National,Ecuador,Primer dia del ano,False,93.14,1.0
2,2,2013-01-01,1,BEAUTY,0.000,0,Quito,Pichincha,D,13,Holiday,National,Ecuador,Primer dia del ano,False,93.14,1.0
3,3,2013-01-01,1,BEVERAGES,0.000,0,Quito,Pichincha,D,13,Holiday,National,Ecuador,Primer dia del ano,False,93.14,1.0
4,4,2013-01-01,1,BOOKS,0.000,0,Quito,Pichincha,D,13,Holiday,National,Ecuador,Primer dia del ano,False,93.14,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3054343,3000883,2017-08-15,9,POULTRY,438.133,0,Quito,Pichincha,B,6,Holiday,Local,Riobamba,Fundacion de Riobamba,False,47.57,1.0
3054344,3000884,2017-08-15,9,PREPARED FOODS,154.553,1,Quito,Pichincha,B,6,Holiday,Local,Riobamba,Fundacion de Riobamba,False,47.57,1.0
3054345,3000885,2017-08-15,9,PRODUCE,2419.729,148,Quito,Pichincha,B,6,Holiday,Local,Riobamba,Fundacion de Riobamba,False,47.57,1.0
3054346,3000886,2017-08-15,9,SCHOOL AND OFFICE SUPPLIES,121.000,8,Quito,Pichincha,B,6,Holiday,Local,Riobamba,Fundacion de Riobamba,False,47.57,1.0


In [ ]:
#Replacing NaN of non-holiday with Normal in case we need it for visualization
train_data['holiday_type'] = train_data['holiday_type'].fillna("Normal")

In [ ]:
#Classifying as Holiday and Non-Holiday
train_data['category_type'] = train_data.apply(
    lambda row: 'non-holiday'
      if ((row['holiday_type'] == 'Holiday' and row['transferred'])
        or (row['holiday_type'] == 'Normal' and ~(row['date'].day_name() in ['Saturday', 'Sunday'])))
      else ('holiday' if ((row['holiday_type'] in ['Holiday', 'Additional','Event', 'Transfer', 'Bridge', 'Work Day'])
        or (row['holiday_type'] == 'Normal' and (row['date'].day_name() in ['Saturday', 'Sunday'])))
                      else 'non-holiday'), axis=1)

In [ ]:
def create_time_feature(df):
    df['dayofmonth'] = df['date'].dt.day
    df['dayofweek'] = df['date'].dt.dayofweek
    df['quarter'] = df['date'].dt.quarter
    df['month'] = df['date'].dt.month
    df['year'] = df['date'].dt.year
    return df

In [ ]:
#dateTime
test_data['date'] = pd.to_datetime(test_data['date'])

In [ ]:
train_data = create_time_feature(train_data)
test_data = create_time_feature(test_data)

train_data.head()

,id,date,store_nbr,family,sales,onpromotion,city,state,type_x,cluster,...,description,transferred,oil_price,day_of_week,category_type,dayofmonth,dayofweek,quarter,month,year
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0,Quito,Pichincha,D,13,...,Primer dia del ano,False,93.14,1.0,holiday,1,1,1,1,2013
1,1,2013-01-01,1,BABY CARE,0.0,0,Quito,Pichincha,D,13,...,Primer dia del ano,False,93.14,1.0,holiday,1,1,1,1,2013
2,2,2013-01-01,1,BEAUTY,0.0,0,Quito,Pichincha,D,13,...,Primer dia del ano,False,93.14,1.0,holiday,1,1,1,1,2013
3,3,2013-01-01,1,BEVERAGES,0.0,0,Quito,Pichincha,D,13,...,Primer dia del ano,False,93.14,1.0,holiday,1,1,1,1,2013
4,4,2013-01-01,1,BOOKS,0.0,0,Quito,Pichincha,D,13,...,Primer dia del ano,False,93.14,1.0,holiday,1,1,1,1,2013


In [ ]:
train_data_forModel = train_data.copy()
train_data_forModel.head()

,id,date,store_nbr,family,sales,onpromotion,city,state,type_x,cluster,...,description,transferred,oil_price,day_of_week,category_type,dayofmonth,dayofweek,quarter,month,year
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0,Quito,Pichincha,D,13,...,Primer dia del ano,False,93.14,1.0,holiday,1,1,1,1,2013
1,1,2013-01-01,1,BABY CARE,0.0,0,Quito,Pichincha,D,13,...,Primer dia del ano,False,93.14,1.0,holiday,1,1,1,1,2013
2,2,2013-01-01,1,BEAUTY,0.0,0,Quito,Pichincha,D,13,...,Primer dia del ano,False,93.14,1.0,holiday,1,1,1,1,2013
3,3,2013-01-01,1,BEVERAGES,0.0,0,Quito,Pichincha,D,13,...,Primer dia del ano,False,93.14,1.0,holiday,1,1,1,1,2013
4,4,2013-01-01,1,BOOKS,0.0,0,Quito,Pichincha,D,13,...,Primer dia del ano,False,93.14,1.0,holiday,1,1,1,1,2013


In [ ]:
train_data_forModel = train_data_forModel.drop(columns={'id','city','state','type_x','cluster','description', 'locale','locale_name','transferred','holiday_type','day_of_week'})

In [ ]:
train_data_forModel.head()

,date,store_nbr,family,sales,onpromotion,oil_price,category_type,dayofmonth,dayofweek,quarter,month,year
0,2013-01-01,1,AUTOMOTIVE,0.0,0,93.14,holiday,1,1,1,1,2013
1,2013-01-01,1,BABY CARE,0.0,0,93.14,holiday,1,1,1,1,2013
2,2013-01-01,1,BEAUTY,0.0,0,93.14,holiday,1,1,1,1,2013
3,2013-01-01,1,BEVERAGES,0.0,0,93.14,holiday,1,1,1,1,2013
4,2013-01-01,1,BOOKS,0.0,0,93.14,holiday,1,1,1,1,2013


In [ ]:
train_data_forModel = pd.get_dummies(train_data_forModel, columns=['family', 'category_type'] )

In [ ]:
# Convert only boolean columns to integers
train_data_forModel[train_data_forModel.select_dtypes('bool').columns] = train_data_forModel.select_dtypes('bool').astype(int)


In [ ]:
columns_oilPrediction = ['store_nbr','oil_price', 'onpromotion', 'dayofmonth', 'dayofweek',
       'quarter', 'month', 'year', 'family_AUTOMOTIVE',
       'family_BABY CARE', 'family_BEAUTY', 'family_BEVERAGES', 'family_BOOKS',
       'family_BREAD/BAKERY', 'family_CELEBRATION', 'family_CLEANING',
       'family_DAIRY', 'family_DELI', 'family_EGGS', 'family_FROZEN FOODS',
       'family_GROCERY I', 'family_GROCERY II', 'family_HARDWARE',
       'family_HOME AND KITCHEN I', 'family_HOME AND KITCHEN II',
       'family_HOME APPLIANCES', 'family_HOME CARE', 'family_LADIESWEAR',
       'family_LAWN AND GARDEN', 'family_LINGERIE', 'family_LIQUOR,WINE,BEER',
       'family_MAGAZINES', 'family_MEATS', 'family_PERSONAL CARE',
       'family_PET SUPPLIES', 'family_PLAYERS AND ELECTRONICS',
       'family_POULTRY', 'family_PREPARED FOODS', 'family_PRODUCE',
       'family_SCHOOL AND OFFICE SUPPLIES', 'family_SEAFOOD',
       'category_type_holiday', 'category_type_non-holiday', 'sales']
train_data_forModel_copy = train_data_forModel.reindex(columns=columns_oilPrediction)

In [ ]:
columns_salePrediction = ['store_nbr','sales', 'onpromotion', 'dayofmonth', 'dayofweek',
       'quarter', 'month', 'year', 'family_AUTOMOTIVE',
       'family_BABY CARE', 'family_BEAUTY', 'family_BEVERAGES', 'family_BOOKS',
       'family_BREAD/BAKERY', 'family_CELEBRATION', 'family_CLEANING',
       'family_DAIRY', 'family_DELI', 'family_EGGS', 'family_FROZEN FOODS',
       'family_GROCERY I', 'family_GROCERY II', 'family_HARDWARE',
       'family_HOME AND KITCHEN I', 'family_HOME AND KITCHEN II',
       'family_HOME APPLIANCES', 'family_HOME CARE', 'family_LADIESWEAR',
       'family_LAWN AND GARDEN', 'family_LINGERIE', 'family_LIQUOR,WINE,BEER',
       'family_MAGAZINES', 'family_MEATS', 'family_PERSONAL CARE',
       'family_PET SUPPLIES', 'family_PLAYERS AND ELECTRONICS',
       'family_POULTRY', 'family_PREPARED FOODS', 'family_PRODUCE',
       'family_SCHOOL AND OFFICE SUPPLIES', 'family_SEAFOOD',
       'category_type_holiday', 'category_type_non-holiday', 'oil_price']
train_data_forModel_sales = train_data_forModel.reindex(columns=columns_salePrediction)

In [ ]:
train_data_forModel_copy

,store_nbr,oil_price,onpromotion,dayofmonth,dayofweek,quarter,month,year,family_AUTOMOTIVE,family_BABY CARE,...,family_PET SUPPLIES,family_PLAYERS AND ELECTRONICS,family_POULTRY,family_PREPARED FOODS,family_PRODUCE,family_SCHOOL AND OFFICE SUPPLIES,family_SEAFOOD,category_type_holiday,category_type_non-holiday,sales
0,1,93.14,0,1,1,1,1,2013,1,0,...,0,0,0,0,0,0,0,1,0,0.000
1,1,93.14,0,1,1,1,1,2013,0,1,...,0,0,0,0,0,0,0,1,0,0.000
2,1,93.14,0,1,1,1,1,2013,0,0,...,0,0,0,0,0,0,0,1,0,0.000
3,1,93.14,0,1,1,1,1,2013,0,0,...,0,0,0,0,0,0,0,1,0,0.000
4,1,93.14,0,1,1,1,1,2013,0,0,...,0,0,0,0,0,0,0,1,0,0.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3054343,9,47.57,0,15,1,3,8,2017,0,0,...,0,0,1,0,0,0,0,1,0,438.133
3054344,9,47.57,1,15,1,3,8,2017,0,0,...,0,0,0,1,0,0,0,1,0,154.553
3054345,9,47.57,148,15,1,3,8,2017,0,0,...,0,0,0,0,1,0,0,1,0,2419.729
3054346,9,47.57,8,15,1,3,8,2017,0,0,...,0,0,0,0,0,1,0,1,0,121.000


Predicting for Oil Sales

In [ ]:
# trainfor_oilPrediction = train_data_forModel_oil.copy()
# trainfor_oilPrediction.columns

In [ ]:
# # trainfor_oilPrediction = trainfor_oilPrediction.loc[:,['store_nbr','date', 'dayofmonth', 'dayofweek', 'quarter', 'month', 'year','category_type_holiday', 'category_type_non-holiday', 'sales', 'oil_price']]

# # trainfor_oilPrediction = trainfor_oilPrediction.groupby(['date', 'dayofmonth', 'dayofweek', 'quarter', 'month', 'year', 'category_type_holiday', 'category_type_non-holiday', 'oil_price'])['sales'].sum().reset_index()
# trainfor_oilPrediction = trainfor_oilPrediction.sort_values('date')
# trainfor_oilPrediction = trainfor_oilPrediction.reset_index(drop=True)

# trainfor_oilPrediction

In [ ]:
# # X = trainfor_oilPrediction[['store_nbr', 'dayofmonth', 'dayofweek', 'quarter', 'month', 'year','category_type_holiday', 'category_type_non-holiday', 'sales' ]]  # Features
# y = trainfor_oilPrediction['oil_price']
# X = trainfor_oilPrediction.drop(['date', 'oil_price'], axis=1)

In [ ]:
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

XGBRegressor

In [ ]:
# !pip install xgboost
# from xgboost import XGBRegressor
# from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# # Initialize the XGBoost regressor
# xgb_model = XGBRegressor(objective='reg:squarederror', random_state=42)

# # Train the model on the training data
# xgb_model.fit(X_train, y_train)

In [ ]:
# # Make predictions on the test set
# y_pred = xgb_model.predict(X_test)


In [ ]:
# # Save the model
# xgb_model.save_model("xgb_oil_price_model.json")

In [ ]:
# # Evaluate the model
# mse = mean_squared_error(y_test, y_pred)
# mae = mean_absolute_error(y_test, y_pred)
# r2 = r2_score(y_test, y_pred)

# print("Mean Squared Error:", mse)
# print("Mean Absolute Error:", mae)
# print("R^2 Score:", r2)

In [ ]:
# import numpy as np

# # If you have MSE, use np.sqrt() to get RMSE
# rmse = np.sqrt(mse)
# print("Root Mean Squared Error:", rmse)


RandomForestRegressor

In [ ]:
# import numpy as np
# from sklearn.ensemble import RandomForestRegressor
# from sklearn.metrics import mean_squared_error

# model = RandomForestRegressor(n_estimators=30, random_state=42)
# model.fit(X_train, y_train)
# y_pred = model.predict(X_test)

# rmse = np.sqrt(mean_squared_error(y_test, y_pred))
# print("Root Mean Squared Error:", rmse)

In [ ]:
# import joblib

# # Save the model to a file
# filename = 'random_forest_model.sav'
# joblib.dump(model, filename)

Linear Regression

In [ ]:
# from sklearn.linear_model import LinearRegression

# model = LinearRegression()
# model.fit(X_train, y_train)
# y_pred = model.predict(X_test)

# rmse = np.sqrt(mean_squared_error(y_test, y_pred))
# print("Root Mean Squared Error:", rmse)

Predict for Oil Price for Test data

In [ ]:
test_data_forModel = test_data.copy()

In [ ]:
#dateTime
test_data_forModel['date'] = pd.to_datetime(test_data_forModel['date'])

In [ ]:
test_data_forModel = pd.merge(test_data_forModel, holiday_data, on='date', how='left')

In [ ]:
test_data_forModel.rename(columns={'type': 'holiday_type'}, inplace=True)

In [ ]:
#Replacing NaN of non-holiday with Normal in case we need it for visualization
test_data_forModel['holiday_type'] = test_data_forModel['holiday_type'].fillna("Normal")

In [ ]:
test_data_forModel = pd.merge(test_data_forModel, oil_data, on='date',how='left')
test_data_forModel['dcoilwtico'] = test_data_forModel['dcoilwtico'].bfill()
test_data_forModel.rename(columns={'dcoilwtico': 'oil_price'}, inplace=True)
test_data_forModel.head()

,id,date,store_nbr,family,onpromotion,dayofmonth,dayofweek,quarter,month,year,holiday_type,locale,locale_name,description,transferred,oil_price,day_of_week
0,3000888,2017-08-16,1,AUTOMOTIVE,0,16,2,3,8,2017,Normal,NaN,NaN,NaN,NaN,46.8,2.0
1,3000889,2017-08-16,1,BABY CARE,0,16,2,3,8,2017,Normal,NaN,NaN,NaN,NaN,46.8,2.0
2,3000890,2017-08-16,1,BEAUTY,2,16,2,3,8,2017,Normal,NaN,NaN,NaN,NaN,46.8,2.0
3,3000891,2017-08-16,1,BEVERAGES,20,16,2,3,8,2017,Normal,NaN,NaN,NaN,NaN,46.8,2.0
4,3000892,2017-08-16,1,BOOKS,0,16,2,3,8,2017,Normal,NaN,NaN,NaN,NaN,46.8,2.0


In [ ]:
#Classifying as Holiday and Non-Holiday
test_data_forModel['category_type'] = test_data_forModel.apply(
    lambda row: 'non-holiday'
      if ((row['holiday_type'] == 'Holiday' and row['transferred'])
        or (row['holiday_type'] == 'Normal' and ~(row['date'].day_name() in ['Saturday', 'Sunday'])))
      else ('holiday' if ((row['holiday_type'] in ['Holiday', 'Additional','Event', 'Transfer', 'Bridge', 'Work Day'])
        or (row['holiday_type'] == 'Normal' and (row['date'].day_name() in ['Saturday', 'Sunday'])))
                      else 'non-holiday'), axis=1)

In [ ]:
test_data_forModel = test_data_forModel.drop(columns={'description','date','holiday_type','locale','locale_name','transferred'})
test_data_forModel

,id,store_nbr,family,onpromotion,dayofmonth,dayofweek,quarter,month,year,oil_price,day_of_week,category_type
0,3000888,1,AUTOMOTIVE,0,16,2,3,8,2017,46.80,2.0,non-holiday
1,3000889,1,BABY CARE,0,16,2,3,8,2017,46.80,2.0,non-holiday
2,3000890,1,BEAUTY,2,16,2,3,8,2017,46.80,2.0,non-holiday
3,3000891,1,BEVERAGES,20,16,2,3,8,2017,46.80,2.0,non-holiday
4,3000892,1,BOOKS,0,16,2,3,8,2017,46.80,2.0,non-holiday
...,...,...,...,...,...,...,...,...,...,...,...,...
28507,3029395,9,POULTRY,1,31,3,3,8,2017,47.26,3.0,non-holiday
28508,3029396,9,PREPARED FOODS,0,31,3,3,8,2017,47.26,3.0,non-holiday
28509,3029397,9,PRODUCE,1,31,3,3,8,2017,47.26,3.0,non-holiday
28510,3029398,9,SCHOOL AND OFFICE SUPPLIES,9,31,3,3,8,2017,47.26,3.0,non-holiday


In [ ]:
test_data_forModel = pd.get_dummies(test_data_forModel, columns=['family', 'category_type'])

In [ ]:
test_data_forModel[test_data_forModel.select_dtypes('bool').columns] = test_data_forModel.select_dtypes('bool').astype(int)


In [ ]:
order_columns = ['store_nbr','oil_price', 'onpromotion', 'dayofmonth', 'dayofweek',
       'quarter', 'month', 'year', 'family_AUTOMOTIVE',
       'family_BABY CARE', 'family_BEAUTY', 'family_BEVERAGES', 'family_BOOKS',
       'family_BREAD/BAKERY', 'family_CELEBRATION', 'family_CLEANING',
       'family_DAIRY', 'family_DELI', 'family_EGGS', 'family_FROZEN FOODS',
       'family_GROCERY I', 'family_GROCERY II', 'family_HARDWARE',
       'family_HOME AND KITCHEN I', 'family_HOME AND KITCHEN II',
       'family_HOME APPLIANCES', 'family_HOME CARE', 'family_LADIESWEAR',
       'family_LAWN AND GARDEN', 'family_LINGERIE', 'family_LIQUOR,WINE,BEER',
       'family_MAGAZINES', 'family_MEATS', 'family_PERSONAL CARE',
       'family_PET SUPPLIES', 'family_PLAYERS AND ELECTRONICS',
       'family_POULTRY', 'family_PREPARED FOODS', 'family_PRODUCE',
       'family_SCHOOL AND OFFICE SUPPLIES', 'family_SEAFOOD',
       'category_type_holiday', 'category_type_non-holiday']
test_data_forModel_sales = test_data_forModel.reindex(columns=order_columns)

In [ ]:
test_data_forModel_sales

,store_nbr,oil_price,onpromotion,dayofmonth,dayofweek,quarter,month,year,family_AUTOMOTIVE,family_BABY CARE,...,family_PERSONAL CARE,family_PET SUPPLIES,family_PLAYERS AND ELECTRONICS,family_POULTRY,family_PREPARED FOODS,family_PRODUCE,family_SCHOOL AND OFFICE SUPPLIES,family_SEAFOOD,category_type_holiday,category_type_non-holiday
0,1,46.80,0,16,2,3,8,2017,1,0,...,0,0,0,0,0,0,0,0,0,1
1,1,46.80,0,16,2,3,8,2017,0,1,...,0,0,0,0,0,0,0,0,0,1
2,1,46.80,2,16,2,3,8,2017,0,0,...,0,0,0,0,0,0,0,0,0,1
3,1,46.80,20,16,2,3,8,2017,0,0,...,0,0,0,0,0,0,0,0,0,1
4,1,46.80,0,16,2,3,8,2017,0,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28507,9,47.26,1,31,3,3,8,2017,0,0,...,0,0,0,1,0,0,0,0,0,1
28508,9,47.26,0,31,3,3,8,2017,0,0,...,0,0,0,0,1,0,0,0,0,1
28509,9,47.26,1,31,3,3,8,2017,0,0,...,0,0,0,0,0,1,0,0,0,1
28510,9,47.26,9,31,3,3,8,2017,0,0,...,0,0,0,0,0,0,1,0,0,1


In [ ]:
# # Make predictions on the test set
# y_pred_forTestData = model.predict(test_data_forModel_oil)
# y_pred_forTestData = np.round(y_pred_forTestData, 2)

# print(y_pred_forTestData)

In [ ]:
# test_data_forModel_oil['oil_price'] = y_pred_forTestData
# test_data_forModel_oil

In [ ]:
y = train_data_forModel_copy.sales

X_train_col = train_data_forModel_copy.drop('sales', axis=1)
X_train_col

,store_nbr,oil_price,onpromotion,dayofmonth,dayofweek,quarter,month,year,family_AUTOMOTIVE,family_BABY CARE,...,family_PERSONAL CARE,family_PET SUPPLIES,family_PLAYERS AND ELECTRONICS,family_POULTRY,family_PREPARED FOODS,family_PRODUCE,family_SCHOOL AND OFFICE SUPPLIES,family_SEAFOOD,category_type_holiday,category_type_non-holiday
0,1,93.14,0,1,1,1,1,2013,1,0,...,0,0,0,0,0,0,0,0,1,0
1,1,93.14,0,1,1,1,1,2013,0,1,...,0,0,0,0,0,0,0,0,1,0
2,1,93.14,0,1,1,1,1,2013,0,0,...,0,0,0,0,0,0,0,0,1,0
3,1,93.14,0,1,1,1,1,2013,0,0,...,0,0,0,0,0,0,0,0,1,0
4,1,93.14,0,1,1,1,1,2013,0,0,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3054343,9,47.57,0,15,1,3,8,2017,0,0,...,0,0,0,1,0,0,0,0,1,0
3054344,9,47.57,1,15,1,3,8,2017,0,0,...,0,0,0,0,1,0,0,0,1,0
3054345,9,47.57,148,15,1,3,8,2017,0,0,...,0,0,0,0,0,1,0,0,1,0
3054346,9,47.57,8,15,1,3,8,2017,0,0,...,0,0,0,0,0,0,1,0,1,0


In [ ]:
# xgb_model_test_sale = XGBRegressor(objective='reg:squarederror', random_state=42)
# xgb_model_test_sale.fit(X_train_col, y)

In [ ]:
# y_pred = xgb_model_test_sale.predict(test_data_forModel_oil)
# y_pred

In [ ]:
#Define a random forest model
rf_model = RandomForestRegressor(random_state=42)
rf_model.fit(X_train_col,y)

RandomForestRegressor(random_state=42)

In [ ]:
predictions = rf_model.predict(test_data_forModel_sales)
predictions

array([   4.26      ,    0.        ,    5.71      , ..., 1264.73332   ,
        131.38      ,   14.39130996])

Version 2 Fine Tuning Random Forest

In [ ]:
# from sklearn.model_selection import GridSearchCV
# from sklearn.ensemble import RandomForestRegressor
# from sklearn.metrics import make_scorer, mean_squared_error
# import numpy as np

# # Define the model
# rf = RandomForestRegressor(random_state=42)

# # Create the parameter grid
# param_grid = {
#     'n_estimators': [100, 200],
#     'max_depth': [10, 20],
#     'min_samples_split': [2, 5, 10],
#     'min_samples_leaf': [2, 4]
# }

# # Define a custom scorer
# scorer = make_scorer(mean_squared_error, greater_is_better=False, squared=False)

# # Instantiate the grid search model
# grid_search_rFModel_forPrice = GridSearchCV(estimator=rf, param_grid=param_grid, cv=3, n_jobs=-1, verbose=2, scoring=scorer)

# # Fit the grid search to the data
# grid_search_rFModel_forPrice.fit(X_train_col, y)

# print("Best parameters found: ", grid_search_rFModel_forPrice.best_params_)
# print("Lowest RMSE found: ", np.sqrt(-grid_search_rFModel_forPrice.best_score_))

In [ ]:
# predictions = grid_search_rFModel_forPrice.predict(test_data_forModel_oil)
# predictions

In [ ]:
# #Using best params to predict for Sales Price

# best_rf_salesPrice = grid_search_rFModel_forPrice.best_estimator_
# predictions = best_rf_salesPrice.predict(test_data_forModel_oil)
# predictions

In [ ]:
# frame = pd.DataFrame(predictions)
# frame.to_csv('predictions.csv')
# frame

In [ ]:
# import joblib

# # Save the model to a file
# filename = 'random_forest_model_forSales.sav'
# joblib.dump(best_rf_salesPrice, filename)

In [ ]:
test_data_forModel['id'] = test_data['id']
test_data_forModel.head()

,id,store_nbr,onpromotion,dayofmonth,dayofweek,quarter,month,year,oil_price,day_of_week,...,family_PERSONAL CARE,family_PET SUPPLIES,family_PLAYERS AND ELECTRONICS,family_POULTRY,family_PREPARED FOODS,family_PRODUCE,family_SCHOOL AND OFFICE SUPPLIES,family_SEAFOOD,category_type_holiday,category_type_non-holiday
0,3000888,1,0,16,2,3,8,2017,46.8,2.0,...,0,0,0,0,0,0,0,0,0,1
1,3000889,1,0,16,2,3,8,2017,46.8,2.0,...,0,0,0,0,0,0,0,0,0,1
2,3000890,1,2,16,2,3,8,2017,46.8,2.0,...,0,0,0,0,0,0,0,0,0,1
3,3000891,1,20,16,2,3,8,2017,46.8,2.0,...,0,0,0,0,0,0,0,0,0,1
4,3000892,1,0,16,2,3,8,2017,46.8,2.0,...,0,0,0,0,0,0,0,0,0,1


In [ ]:
submission = pd.DataFrame({'id': test_data_forModel['id'], 'sales':predictions})
submission.to_csv('submission.csv',index=False)